# 自分で育てた馬を有馬記念に
**説明変数**
* グラウンド
* 馬場の状態
* 距離
* 天候
* どっち周りか
* 騎手の名前

**目的変数**
* タイム

**やることの想定**
* まずはスクレイピングで今回出馬した馬の上記の変数を取り出す。(今回はめんどくさいので1,2,3位の馬の情報のみ抽出することにする)
* その後機械学習によってそれぞれのタイムを機械学習によって推測させる。

In [ ]:
# スクレイピング
from bs4 import BeautifulSoup
# HTTP操作用
import requests 
# サーバーに負担をかけないようにする
import time
# csvを作るために使うよ
import csv
# 考察に使うやつ
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt